<a href="https://colab.research.google.com/github/max2000777/Traitement-de-la-langue-naturelle/blob/main/%5BMIDS%5D_TAL_TP_s%C3%A9ance_3_Analyse_de_sujets_latents_Maxime_BRENNAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copier ce notebook (Fichier>Enregistrer une copie dans Drive) puis travailler sur la copie.
==

# But du TP

Dans ce TP, nous allons faire de l'inférence de sujets latents.
Il s'agit de décrire un corpus de textes à l'aide d'un ensemble de sujets (à déterminer), chaque document étant plus ou moins fortement corrélé avec les différents sujets, et chaque sujet étant plus ou moins fortement corrélé avec les différents mots du vocabulaire.
Plus précisement, nous allons utiliser la bibliothèque scikit-learn pour entraîner puis analyser un modèle de type Latent Semantic Analysis (LSA) et un modèle de type Latent Dirichlet Allocation (LDA).

Nous utiliserons un jeu de données « réelles », composé de sous-titres français de la série Game of Thrones.
La première section du notebook génère aussi un jeu de données artificiel, particulièrement adapté à l'analyse par sujets latents.
Ce jeu de données permet de tester le code et de montrer ce que les différents systèmes d'analyse étudiés peuvent donner dans un cas idéal.
**Si votre code s'exécute sans erreur mais que, au vu des résultats, vous n'êtes pas sûr·e de son bon fonctionnement, testez-le sur le jeu de données artificiel.**

In [ ]:
#use_controlled_dataset = True
use_controlled_dataset = False

# Préparation d'un jeu de données artificiel

Pour vérifier le bon fonctionnement de nos systèmes d'analyse de sujets latents, nous allons utiliser un jeu de données artificiel, composé de textes que nous savons être issus de sujets bien distincts.

Les « sujets » utilisés ici sont les entiers de `0` à `n_artificial_topics=15`, ou, de manière équivalente, les lettres de l'alphabet de `"A"` (pour `0`) à `"O"` (pour `15`).
À chaque sujet correspond un petit nombre de mots, tous obtenus par répétition de la lettre associée au sujet (ex : « B » et « BBB » pour le sujet 1/B).
Chacun des `n_docs=40` documents est associé à un sujet `main_topic` et est constitué de `doc_length=2000` mots, chacun issu d'un sujet choisi avec une probabilité décroissante de sa distance à `main_topic`.

Par exemple, le huitième document, d'identifiant « doc_C_7 », est associé au topic 2/C : il contient beaucoup de mots en C, un peu moins de mots en B et D, un peu moins de mots en A et E, etc.

Un système d'analyse de sujet devrait être capable de retrouver au moins approximativement les différents sujets utilisés pour générer ce jeu de données artificiel.

In [ ]:
if(use_controlled_dataset):
  import numpy as np

  n_docs = 40
  doc_length = 2000
  n_artificial_topics = 15

  import random

  dataset = []
  for i in range(n_docs):
    main_topic = int(i  * n_artificial_topics / n_docs)
    print(main_topic, end=", ", flush=True)

    tokens = []
    for j in range(doc_length):
      # Selects a topic for the word, likely to be main_topic, or very near.
      topic = main_topic + int(np.random.randn() * 1.5)
      while(topic < 0): topic += n_artificial_topics
      topic = topic % n_artificial_topics

      #
      letter = chr(65 + topic) # 65 corresponds to "A".
      token_length = np.floor(1 + 8 * np.power(np.random.uniform(), 2)) # Between 1 and 9; more likely to be small than large.
      token = letter * int(token_length)

      tokens.append(token)

    dataset.append({"str_id": f"doc_{chr(65+main_topic)}_{i}", "raw_text": " ".join(tokens)})
else:
  print("[controlled dataset not in use]")

In [ ]:
if(use_controlled_dataset):
  print(dataset[7]["str_id"])
  print(dataset[7]["raw_text"]) # Should be composed mainly of words in C, then words in B or D, then words in A or E, etc.
else:
  print("[controlled dataset not in use]")

In [ ]:
if(use_controlled_dataset):
  for episode in dataset:
    print(episode["str_id"], end=", ", flush=True)
    episode["processed_text"] = episode["raw_text"] # No preprocessing for the artificial dataset.
    episode["processed_tokens"] = episode["raw_text"].split()
else:
  print("[controlled dataset not in use]")

In [ ]:
if(use_controlled_dataset):
  print(dataset[0]["str_id"])
  print()
  print(dataset[0]["processed_tokens"][:20])
  print("[…]")
  print(dataset[0]["processed_tokens"][-20:])
else:
  print("[controlled dataset not in use]")

# Préparation du jeu de données réelles

Le but de cette section est de charger les dialogues de la série Game of Thrones dans une liste `dataset` contenant un dictionaire par épisode.
Chaque épisode sera représenté par un dictionnaire indiquant, entre autres,

*   un identifiant (*str_id*) sous la forme « SxxEyy » où xx et yy indiquent respectivement le numéro de saison et d'épisode (dans la saison) de l'épisode ;
*   une liste de tokens (*processed_tokens*), qui sera par la suite convertie en vecteur numérique de type sac-de-mots (« *bag-of-words* »).

## Téléchargement et extraction du jeu de données

In [ ]:
if(not use_controlled_dataset):
  import os
  import urllib # To download files.
  import zipfile # To unzip files.

  if(True): # True for French data, False for English data.
    zip_url = "https://moodle.u-paris.fr/mod/resource/view.php?id=1329463" # fr
  else:
    zip_url = "https://moodle.u-paris.fr/mod/resource/view.php?id=1330372" # en

  data_dirname = "data" # Name of the directory in which the dataset is/will be.

  if(os.path.isdir(data_dirname)):
    print("Dataset found.")
  else:
    # Downloads the dataset.
    tmp = urllib.request.urlretrieve(zip_url)
    filename = tmp[0]
    print(f"Dataset downloaded to '{filename}'.")

    # Extracts the dataset.
    with zipfile.ZipFile(filename, 'r') as zip_ref:
      zip_ref.extractall(".")
    assert os.path.isdir(data_dirname)
    print(f"Dataset extracted to '{data_dirname}'.")
else:
  print("[controlled dataset in use]")

## Lecture du jeu de données

Le jeu de données contient un fichier SRT par épisode de la série Game of Thrones, groupés par saison (un sous-dossier par saison).
(La syntaxe des fichiers SRT est décrite ici : https://docs.fileformat.com/video/srt/)

Cette sous-section crée la liste `dataset` dans laquelle chaque épisode est représenté par un dictionnaire indiquant, pour l'instant, son identifiant (*str_id*) et une chaîne de caractères contenant le contenu des sous-titre de l'épisode (*raw_text*).

In [ ]:
if(not use_controlled_dataset):
  dataset = [] # This will be a list of dictionaries.

  # file_path: str
  # Returns a string.
  def read_srt_file(file_path):
      lines = []
      with open(file_path) as f:
          c = True
          while(c):
              s = f.readline() # This should be the end of the file, an empty line or a number.
              if(s == ""): c = False # The end of the file has been reached.
              if(s.strip() == ""): continue # End of the file or empty line.

              f.readline() # We can throw away the timing that follows.

              # All the next non-empty lines are character lines.
              s = f.readline().strip()
              while(s != ""):
                  lines.append(s)
                  s = f.readline().strip()

      return ' '.join(lines)

  data_dirname = "data"
  for path, dirs, files in os.walk(data_dirname): # Iterates through every subdirectories.
      path_parts = path.split(os.path.sep)
      if(len(path_parts) == 1): continue # There is no subtitle file in the root directory.
      season = path_parts[1]
      #print(season)
      for file in files:
          file_parts = file.split(".")
          episode = file_parts[0]
          str_id = f"{season}-{episode}"
          print(str_id, end= ", ")

          file_path = os.path.join(path, file)
          #print(file_path)

          dataset.append({"str_id": str_id, "raw_text": read_srt_file(file_path)})
else:
  print("[controlled dataset in use]")

In [ ]:
if(not use_controlled_dataset):
  dataset = sorted(dataset, key=(lambda x: x["str_id"])) # Sorts the episode chronologically (via their season and episode number).

  for episode in dataset: print(episode["str_id"], end=", ")
else:
  print("[controlled dataset in use]")

In [ ]:
if(not use_controlled_dataset):
  print(f'{dataset[0]["str_id"]}:')
  print(dataset[0]["raw_text"][:200]) # Beginning of the first episode.
  print("[…]")
  print(dataset[0]["raw_text"][-200:]) # End of the first episode.
else:
  print("[controlled dataset in use]")

## Pré-traitement du texte

Cette sous-section enrichit le dictionnaire représentant chaque épisode avec, notamment, la liste de tokens (*processed_tokens*) qui sera ensuite convertie en vecteur numérique de type sac-de-mots (« *bag-of-words* »).
Cette liste est obtenue des sous-titres de l'épisode en plusieurs étapes : (i) normalisation/simplification du texte, (ii) tokenisation, (iii) filtrage des mots vides (« *stop words* », c.-à-d. les mots non pertinents pour le problème car n'ayant que peu de valeur sémantique, comme les articles « le/la/… » ou les conjonctions « mais/ou/et/donc/… »).

Le bloc suivant définit, si nécessaire, l'ensemble de mots vides à utiliser (`stopwords`).

In [ ]:
if(not use_controlled_dataset):
  filter_stopwords = True
  #filter_stopwords = False

  import nltk

  if(filter_stopwords):
    try:
      print(f"NLTK stop words: {nltk.corpus.stopwords.words('french')}") # This might fail if "stopwords" is missing.
    except:
      nltk.download('stopwords')
      print(f"NLTK stop words: {nltk.corpus.stopwords.words('french')}")

    stopwords = set()
    stopwords.update(set(nltk.corpus.stopwords.words("french")))

    # Additional stop words.
    stopwords.update({"a", "si", "plus", "fait", "faire", "ça", "tout", "tous", "toute", "toutes", "ce", "celui", "ceux", "celle", "celles", "son", "sa", "ses", "leur", "leurs", "tu", "dit", "oui", "non", "si", "alors", "ne", "être", "avoir", "faut", "veux", "i", "ici", "là", "où", "quand", "veut", "peut", "il", "ils", "elle", "elles", "mais", "ou", "et", "donc", "car"})

    print(f"Stop words used: {stopwords}")
else:
  print("[controlled dataset in use]")

Le bloc suivant s'assure de la disponibilité du modèle de tokenisation à utiliser (`nltk.word_tokenize`).

In [ ]:
if(not use_controlled_dataset):
  try:
    print(nltk.word_tokenize("NLTK tokeniser ready.")) # This might fail if "punkt" is missing.
  except:
    # Modifications in 2025/01/23.
    #nltk.download('punkt') # Necessary to use nltk.word_tokenize.
    nltk.download('punkt_tab') # Necessary to use nltk.word_tokenize.

    print(nltk.word_tokenize("NLTK tokeniser ready."))
else:
  print("[controlled dataset in use]")

Le bloc suivant définit, si nécessaire, le modèle de racinisation (« *stemming* ») à utiliser (`stemmer`).


In [ ]:
if(not use_controlled_dataset):
  stem_words = True
  stem_words = False

  if(stem_words):
    stemmer = nltk.stem.snowball.FrenchStemmer() # https://www.nltk.org/api/nltk.stem.snowball.html
else:
  print("[controlled dataset in use]")

Le bloc suivant définit la fonction utilisée pour pré-traiter le texte de chaque épisode. C'est elle qui effectue les trois étapes, mentionnées plus haut, de normalisation/simplification, tokenisation, et filtrage.

Concernant l'étape de normalisation/simplification, la fonction doit au minimum passer tous les caractères en bas de casse et supprimer la ponctuation.

Des opérations supplémentaires sont envisageables, au sein de l'étape de normalisation/simplification, comme après la tokenisation.

In [ ]:
if(not use_controlled_dataset):
  import re # For regexes. https://docs.python.org/3/library/re.html https://docs.python.org/3/howto/regex.html

  # text: str
  # Returns a pair composed of (i) a string (the text just before tokenization) and (ii) a list of tokens (i.e. strings).
  def preprocess(text):
      # (i) Normalisation/simplification step
      tmp = text
      ## TODO
      tmp = re.sub(r'[^\w\s]', ' ', tmp) # Replaces any non-alphanumeric character (\w) and non-whitespace character (\s) with a space.
      tmp = re.sub(r'[0-9]', ' ', tmp) # Replaces any numeric character with a space.
      tmp = tmp.lower() # Lower cases the string
      #tmp = re.sub(r'\s{2,}', ' ', tmp) # Replaces any duplicate spaces with a single space.
      processed_text = tmp

      # (ii) Tokenisation step
      ## TODO
      tokens = nltk.word_tokenize(processed_text)

      # (iii) Filtering step
      ## TODO
      if(filter_stopwords): tokens = [token for token in tokens if(token not in stopwords)]

      # (bonus) Stemming step
      if(stem_words): tokens = [stemmer.stem(token) for token in tokens]

      return (processed_text, tokens)

  # Test
  print(preprocess("Yo Jon Snow, comment ça ?  Plutôt bien et vous ? Comptez jusqu'à 3, s'il vous plaît. 1, 2, 3. Nickel."))
else:
  print("[controlled dataset in use]")

Le bloc suivant effectue le pré-traitement des épisodes implémenté par la fonction définie ci-dessus.

In [ ]:
if(not use_controlled_dataset):
  for episode in dataset:
    print(episode["str_id"], end=", ", flush=True)

    (text, tokens) = preprocess(episode["raw_text"])
    episode["processed_text"] = text
    episode["processed_tokens"] = tokens
else:
  print("[controlled dataset in use]")

In [ ]:
if(not use_controlled_dataset):
  print(dataset[0]["str_id"])
  print()
  print(dataset[0]["processed_tokens"][:20])
  print("[…]")
  print(dataset[0]["processed_tokens"][-20:])
else:
  print("[controlled dataset in use]")

In [ ]:
print(dataset[0].keys())

# Création de la matrice de comptage

Le but de cette section est de créer une matrice de comptage `matrix` indiquant, pour un ensemble de formes choisies (c.-à-d. type de tokens), pour chaque document, le nombre d'occurrences de chaque forme dans le document.

Nous allons procéder en trois étapes : (i) comptage par document des formes, (ii) création d'un vocabulaire incluant ou non un forme en fonction de sa fréquence de documents (c.-à-d. la proportion d'épisodes dans lesquels elle apparaît ; le but étant de filtrer les mots trop rares et les mots trop fréquents), (iii) création de la matrice de comptage, restreinte au vocabulaire fixé.

## Comptage par document des formes

Le bloc suivant compte (i) pour chaque forme, le nombre de documents dans laquelle est apparaît, et (ii) pour chaque document, le nombre d'occurrences de chaque forme.

In [ ]:
from collections import Counter # https://docs.python.org/3/library/collections.html#collections.Counter

document_form_counts = Counter() # Form each form, the number of documents it occurs in.
for episode in dataset:
    print(episode["str_id"], end=", ", flush=True)

    episode["counts"] = Counter(episode["processed_tokens"]) # For each form, the number of its occurences in the document.
    document_form_counts.update(set(episode["processed_tokens"]))
print()

print(document_form_counts)

In [ ]:
print(dataset[0].keys())

## Création d'un vocabulaire

Il s'agit ici de créer le vocabulaire utilisé par la suite, contenant les formes dont la fréquence de document est supérieure à une limite inférieure `min_df` et inférieure à une limite supérieure `max_df`.
Ce vocabulaire se présentera sous la forme d'un dictionnaire `form2id`, associant à chaque forme un identifiant entier, et, inversement, d'une liste `id2form`, associant à chaque entier la forme correspondante.

In [ ]:
max_df = 0.9 # Upper limit for the document frequency of a form.
min_df = 0.06 # Lower limit for the document frequency of a form.

# TODO
form2id = {} # From form (str) to id (int).
id2form = [] # From id (int) to form (str).
Nb_doc=len(dataset)

for forme, comptage in document_form_counts.items():
  episode_frequence= comptage/Nb_doc
  if episode_frequence<=max_df and episode_frequence>=min_df :
    form2id[forme]=len(form2id)
    id2form.append(forme)


print(f"Number of forms: {len(id2form)}")

In [ ]:
print(form2id)

## Création de la matrice de comptage

Créer une matrice de comptage sous forme d'un tableau Numpy bidimensionnel.
Les lignes doivent être indicées par les documents, les colonnes par les formes.

In [ ]:
import numpy as np

# TODO
matrix = np.zeros((Nb_doc,len(form2id)))
print(matrix.shape)
for i  in range(0,len(dataset)):
  for forme,comptage in dataset[i]['counts'].items():
    if forme in id2form:
      matrix[i,form2id[forme]]=comptage

print(matrix)

doc_lengths = matrix.sum(axis=1) # For each document, its length.
term_frequency = matrix.sum(axis=0) # For each form, its frequency (count).

# Latent Semantic Analysis (LSA)

## Entraînement d'un modèle

In [ ]:
%%time

from sklearn.decomposition import TruncatedSVD # https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

lsa_n_topics = 15
assert lsa_n_topics <= len(dataset) # With LSA, their cannot be more topics than documents.
lsa_model = TruncatedSVD(n_components=lsa_n_topics, n_iter=10)

print("Fitting the model…", end="", flush=True)
lsa_model.fit(X=matrix)
print(" Done!")

## Analyse des paramètres

In [ ]:
topic_form_corr = lsa_model.components_ # The V^t matrix. Contains a description of each topic in terms of the forms (positive/negative values are interpreted as positive/negative correlations).
print(topic_form_corr.shape)
print(topic_form_corr)

In [ ]:
document_topic_corr = lsa_model.transform(matrix) # (U * Σ). Contains a description of each document in terms of the topics (positive/negative values are interpreted as positive/negative correlations).
print(document_topic_corr.shape)
print(document_topic_corr)

Il s'agit ici, pour chaque sujet, de retrouver les formes de poids maximal et les formes de poids minimal.

In [ ]:
n_forms = 10
def lsa_show_topic(topic_id):
    print(f"Topic n°{topic_id}:")

    topic_vector = topic_form_corr[topic_id]
    #print(len(topic_vector)) # From id (int) to score (float).

    # TODO
    positive_corr_id = []
    negative_corr_id = []
    for i in range(0,len(topic_vector)):
      if topic_vector[i]>0:
        positive_corr_id.append(i)
      else:
        negative_corr_id.append(i)
    print(f"positive correlation: {[(id2form[i], topic_vector[i]) for i in positive_corr_id]}")
    print(f"negative correlation: {[(id2form[i], topic_vector[i]) for i in negative_corr_id]}")

    print()

for topic_id in range(lsa_n_topics): lsa_show_topic(topic_id) # The first topics are the most important.

# Latent Dirichlet Allocation

## Entraînement d'un modèle

In [ ]:
%%time

from sklearn.decomposition import LatentDirichletAllocation

lda_n_topics = 15
lda_model = LatentDirichletAllocation(n_components=lda_n_topics, max_iter=20, n_jobs=-1)

print("Fitting the model…", end="", flush=True)
lda_model.fit(X=matrix)
print(" Done!")

topic_term_dists = lda_model.components_ / np.expand_dims(lda_model.components_.sum(axis=1), axis=-1) # Contains, for each topic, the probability distribution of generation over forms.
doc_topic_dists = lda_model.transform(matrix) # Contains, for each document, the probability distribution over topics.

## Analyse des paramètres

Il s'agit ici, pour chaque sujet, de retrouver les formes de probabilité maximale.

Noter qu'il est possible que certains sujets soient très « étalés », c'est-à-dire aient une distribution assez uniforme sur le vocabulaire.
Ces sujets ne sont pas très informatifs et on pourra vouloir les ignorer par la suite.

In [ ]:
n_forms = 20
def lda_show_topic(topic_id):
    print(f"Topic n°{topic_id}:")

    topic_vector = topic_term_dists[topic_id]
    print(topic_vector) # From id (int) to score (float).

    # TODO
    significant_id = ()

    print([(id2form[i], topic_vector[i]) for i in significant_id])

    print()

for topic_id in range(lda_n_topics): lda_show_topic(topic_id)

On regarde ici, pour chaque document, les principaux sujets qui lui sont associés (c.-à-d., qui, d'après le modèle, interviennent dans la génération du document).
L'ensemble, sur tous les documents, des sujets principaux, est calculé dans `major_topics`.

In [ ]:
major_topics = set()

for (document_id, document_vector) in enumerate(doc_topic_dists):
    #print(f"Document n°{document_id}:")
    print(f"{dataset[document_id]['str_id']}:")

    #print(document_vector) # From id (int) to score (float).

    sorted_id = document_vector.argsort() # https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
    sorted_id = np.flip(sorted_id) # https://numpy.org/doc/stable/reference/generated/numpy.flip.html
    major_id = sorted_id[:n_forms]
    major_id = [i for i in major_id if(document_vector[i] > (0.1 * document_vector.max()))]
    #major_id = [i for i in major_id if(document_vector[i] > (1.1 * document_vector.min()))]

    print(major_id)
    print([document_vector[i] for i in major_id]) # Prints the probability within this document of each major topic.
    major_topics.update(major_id)

    print()

In [ ]:
print(major_topics)
print()

for topic_id in major_topics: lda_show_topic(topic_id)

On observe ici la présence de chaque sujet de `major_topics` dans les différents documents du jeu de données.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.set_size_inches(18.5, 10.5)

time = np.arange(len(dataset))
for i, topic_evolution in enumerate(np.transpose(lda_model.transform(matrix))):
    if(i not in major_topics): continue
    ax.plot(time, topic_evolution, label=f"topic {i}")

ax.legend(loc='lower right')
plt.show()

On représente ici le modèle LDA, en particulier les sujets inférés, à l'aide de la bibliothèque pyLDAvis (https://pyldavis.readthedocs.io/en/latest/readme.html).

L'interface est essentiellement composée d'un panneau gauche, organisant spatialement les différents sujets, et d'un panneau droit représentant la distribution sur le vocabulaire d'un sujet particulier.
Le panneau droit montre, pour un sujet sélectionné, les formes qui lui sont le plus « pertinentes ».
La notion de pertinence utilisée dépend d'un paramètre λ réglable et est décrite en bas du panneau.
Avec λ=1, les mots les plus pertinents sont ceux qui sont les plus fréquemment générés par ce sujet.
Avec λ=0, les mots les plus pertinents sont ceux qui sont les plus spécifiques à ce sujet.

Noter que la numération des sujets par cet outil n'est pas celle utilisée plus haut.
L'outil organise les sujets par importance décroissante dans le corpus, l'importance d'un sujet étant le nombre de tokens qu'il a généré (remarque : se rappeler le modèle génératif de la LDA).

In [ ]:
try:
  import pyLDAvis
except:
  !pip install pyLDAvis
  #!pip install pyLDAvis==3.3.1
  import pyLDAvis

vis_data = pyLDAvis.prepare(topic_term_dists=topic_term_dists, doc_topic_dists=doc_topic_dists, doc_lengths=doc_lengths, vocab=id2form, term_frequency=term_frequency, mds='mmds') # https://pyldavis.readthedocs.io/en/latest/modules/API.html#pyLDAvis.prepare
pyLDAvis.display(vis_data) # Warning: This tool numbers the topics according to their importance in the corpus (in term of number of tokens). It is possible to recognise a topic by looking at the list of forms displayed for λ=1. The list displayed for λ=0 shows the forms that are the most specific to the selected topic.

# Top2Vec

In [ ]:
try:
  from top2vec import Top2Vec
except:
  !pip install top2vec
  from top2vec import Top2Vec

Top2Vec ne fonctionne pas bien (le code peut même crasher) sur des petits ensembles de documents. C'est pourquoi le code suivant scinde chaque document en parties de moins de `max_tokens=500` tokens.

In [ ]:
import itertools
import numpy as np

max_tokens = 250
documents = [" ".join(tokens) for tokens in itertools.chain(*[np.array_split(episode["processed_tokens"], np.ceil(len(episode["processed_tokens"])/max_tokens)) for episode in dataset])] # list[str]

print(f"from {len(dataset)} to {len(documents)} (shorter) documents")
print()

ids = [0, (len(documents)//2), -1]
for i in ids:
  print(f"Document n°{i}:")
  print(documents[i])
  print(f"(char len: {len(documents[i])})")
  print()

## Entraînement du modèle

In [ ]:
try:
  import torch
  gpu_available = (torch.cuda.device_count() > 0)
except:
  gpu_available = False

if(not gpu_available): print("No GPU available. Neural computations might be very slow. You might be able to make a GPU available by changing the notebook's settings.")
else: print("GPU available.")

In [ ]:
top2vec_model = Top2Vec(documents=documents, ngram_vocab=False, contextual_top2vec=False) # https://top2vec.readthedocs.io/en/latest/api.html#top2vec.top2vec.Top2Vec

## Analyse

Garder à l'esprit que Top2Vec tend à ne donner des résultats intéressants que quand les sujets ne sont pas trop mélangés au sein de chaque document. Ça marche par exemple très bien avec le jeu de données artificiel.

In [ ]:
num_topics = top2vec_model.get_num_topics()
print(f"{num_topics} topics induced.")

print()
topic_words, word_scores, topic_nums = top2vec_model.get_topics(num_topics)
for i in topic_nums:
  print("Top 50 vocabulary item in terms of semantic similarity to the topic n°{i}:")
  print(list(zip(topic_words[i], word_scores[i])))

In [ ]:
for i in topic_nums:
  top2vec_model.generate_topic_wordcloud(i)

In [ ]:
La bibliothèque Top2Vec propose d'autres fonctionnalités.
Plus d'information :
*  sur la page GitHub de Top2Vec, https://github.com/ddangelov/Top2Vec
*  dans la documentation de Top2Vec, https://top2vec.readthedocs.io

Voir aussi les (pré)publications scientifiques :
*   Angelov, Dimo. « Top2Vec: Distributed Representations of Topics ». arXiv, 19 août 2020. https://doi.org/10.48550/arXiv.2008.09470.
*   Angelov, Dimo, et Diana Inkpen. « Topic Modeling: Contextual Token Embeddings Are All You Need ». In Findings of the Association for Computational Linguistics: EMNLP 2024, édité par Yaser Al-Onaizan, Mohit Bansal, et Yun-Nung Chen, 13528‑39. Miami, Florida, USA: Association for Computational Linguistics, 2024. https://doi.org/10.18653/v1/2024.findings-emnlp.790.
